In [2]:
import torch
import pandas as pd

In [3]:
df = pd.read_json('/home/userroot/dev/iTransformer/index_300_keynotes.jsonl', lines=True)

In [4]:
df.shape[0]

9566

In [5]:
t = torch.tensor(df['embeddings']).reshape(df.shape[0],1, -1).squeeze()

In [6]:
df_embeddings = pd.DataFrame(t)

In [7]:
df_embeddings.insert(0, 'date', df['date'])
df_embeddings.insert(1, 'code', df['code'])



In [8]:
code_uniq = df_embeddings['code'].unique()
code_uniq

df_code_idx = pd.DataFrame(code_uniq, columns=['code'])


In [9]:
df_code_idx
dict_arr = {value: index for index, value in enumerate(code_uniq)}
dict_arr

{'sz300750': 0,
 'sh600519': 1,
 'sh601318': 2,
 'sh600036': 3,
 'sz000333': 4,
 'sz000858': 5,
 'sh600900': 6,
 'sh601899': 7,
 'sh601166': 8,
 'sh600276': 9}

In [70]:
import tushare as ts
pro = ts.pro_api()
for i, g in df_embeddings.groupby('code'):
    print(g['code'].iloc[0])
    code = g['code'].iloc[0]
    part1 = code[:2]
    part2 = code[2:]

    idx = dict_arr[code]
    sorted_g = g.sort_values(by='date')
    #sorted_g.insert(2,'idx',idx)
    df_day = sorted_g['date'].apply(lambda x: x.strftime('%Y%m%d'))

    sorted_g['day'] = df_day
    sorted_g.drop('date', axis=1, inplace=True)
    df_day_news = sorted_g.groupby(['code','day']).sum().reset_index()

    min_date = (df_day.min())
    max_date = (df_day.max())
    
    df_kline = pro.daily(ts_code=part2+'.'+part1, start_date=min_date, end_date=max_date)
    df_kline.sort_values(by='trade_date', inplace=True)
    df_kline['idx'] = idx

    df_out = pd.merge(df_kline, df_day_news, left_on='trade_date', right_on='day', how='left')
    df_out.pop('code')
    date = df_out.pop('day')
    #df_out.insert(0, 'code', code)
    idx = df_out.pop('idx')
    df_out.insert(1, 'idx', idx)

    trade_date = df_out.pop('trade_date')
    df_out.insert(0, 'date', trade_date)
    df_out.drop(['ts_code'], axis=1, inplace=True)

    df_out.to_csv(f'./index_300/{code}.csv',index=False)
    #break

sh600036
sh600276
sh600519
sh600900
sh601166
sh601318
sh601899
sz000333
sz000858
sz300750


In [67]:
df_out

,date,idx,open,high,low,close,pre_close,change,pct_chg,vol,...,1526,1527,1528,1529,1530,1531,1532,1533,1534,1535
0,20231030,3,31.01,31.13,30.55,30.90,31.53,-0.63,-1.9981,838500.56,...,0.000009,-0.011626,0.133783,-0.302625,0.102012,0.100707,-0.463758,0.087314,0.292798,0.137456
1,20231031,3,30.74,30.99,30.61,30.76,30.90,-0.14,-0.4531,528015.99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20231101,3,30.80,30.88,30.43,30.45,30.76,-0.31,-1.0078,529897.07,...,0.000001,-0.029133,0.017732,-0.041162,0.013725,0.001377,-0.059778,0.016609,0.050532,0.017220
3,20231102,3,30.60,30.81,30.51,30.68,30.45,0.23,0.7553,364667.71,...,0.000001,-0.013911,0.011546,-0.011610,0.001943,0.002128,-0.037798,0.009851,0.036596,0.031129
4,20231103,3,30.68,31.04,30.60,30.99,30.68,0.31,1.0104,439568.79,...,0.000001,-0.004545,0.013368,-0.010489,0.011712,-0.003563,-0.056975,0.032437,0.049904,0.021996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,20240328,3,32.34,32.50,32.07,32.18,32.30,-0.12,-0.3715,694744.92,...,0.000050,-1.335978,1.472967,0.269933,1.201801,-0.583121,-2.743291,1.374162,2.269065,1.280783
102,20240329,3,32.10,32.64,32.07,32.20,32.18,0.02,0.0622,514001.31,...,0.000031,-0.826183,0.875833,-0.039026,0.738412,-0.413422,-1.299196,0.531837,1.176972,0.448053
103,20240401,3,32.18,32.85,32.18,32.64,32.20,0.44,1.3665,570152.32,...,0.000036,-0.633427,0.391564,-0.389442,0.600449,-0.426999,-2.146443,0.236756,1.008351,0.545960
104,20240402,3,32.70,32.92,32.44,32.55,32.64,-0.09,-0.2757,536618.99,...,0.000037,-1.156892,0.904158,-0.181451,0.524649,-0.547327,-1.878498,0.699713,1.178075,0.913404


In [32]:
pd.merge(df_kline, df_day_news, left_on='trade_date', right_on='day', how='left').columns.to_list()[:20]

['ts_code',
 'trade_date',
 'open',
 'high',
 'low',
 'close',
 'pre_close',
 'change',
 'pct_chg',
 'vol',
 'amount',
 'code',
 'idx',
 'day',
 0,
 1,
 2,
 3,
 4,
 5]

In [15]:
sorted_g.groupby(['code','idx','day']).sum()

0         1         2         3         4     \
code     idx day                                                          
sh600036 3   20231028  0.052496  0.014271  0.004077 -0.016782  0.019934   
             20231029  0.714024  0.065521  0.019565  0.063348  0.147568   
             20231030  1.013874  0.061956  0.136891  0.040366  0.253617   
             20231101  0.088131  0.008092  0.028570 -0.026598  0.018498   
             20231102  0.171141 -0.020110  0.026719  0.001360 -0.012637   
...                         ...       ...       ...       ...       ...   
             20240402  2.721682  0.770165  0.948058 -0.016964  0.488265   
             20240403  1.421373  0.314657  0.574405 -0.103237  0.444065   
             20240404  0.460762  0.045750  0.255937 -0.077255  0.058724   
             20240405  0.479731  0.039856  0.145834  0.141446  0.015230   
             20240406  0.451252  0.059133  0.297304  0.036926  0.125510   

                           5         6         7         8         9     ...  \
code     idx day                                                         ...   
sh600036 3   20231028  0.042626  0.018953 -0.006233  0.114500  0.008610  ...   
             20231029  0.307323  0.302455  0.076689  0.548733  0.206721  ...   
             20231030  0.478124  0.592757  0.043058  0.953349  0.360063  ...   
             20231101  0.047380  0.016645 -0.021242  0.096557  0.024940  ...   
             20231102  0.071702  0.056196 -0.005677  0.138932  0.031272  ...   
...                         ...       ...       ...       ...       ...  ...   
             20240402  1.523463  2.155370  0.326618  1.497572  0.922950  ...   
             20240403  0.677203  1.390551  0.392631  0.720667  0.254676  ...   
             20240404  0.379485  0.308425  0.066692  0.193518  0.013732  ...   
             20240405  0.304184  0.137109  0.130456  0.231094  0.111042  ...   
             20240406  0.252566  0.455302  0.028752  0.130500  0.043152  ...   

                           1526      1527      1528      1529      1530  \
code     idx day                                                          
sh600036 3   20231028  0.000001 -0.025786  0.029685 -0.026005  0.012603   
             20231029  0.000007 -0.069186  0.049365 -0.166468  0.085142   
             20231030  0.000009 -0.011626  0.133783 -0.302625  0.102012   
             20231101  0.000001 -0.029133  0.017732 -0.041162  0.013725   
             20231102  0.000001 -0.013911  0.011546 -0.011610  0.001943   
...                         ...       ...       ...       ...       ...   
             20240402  0.000037 -1.156892  0.904158 -0.181451  0.524649   
             20240403  0.000024 -0.840107  0.418354 -0.111304  0.486526   
             20240404  0.000007 -0.250288  0.195225 -0.011342  0.101750   
             20240405  0.000006 -0.171141  0.116491 -0.172336  0.051579   
             20240406  0.000006 -0.035822  0.282217 -0.056454  0.180779   

                           1531      1532      1533      1534      1535  
code     idx day                                                         
sh600036 3   20231028  0.009661 -0.055642  0.033669  0.036433  0.016002  
             20231029  0.102239 -0.336227  0.088117  0.194803  0.150933  
             20231030  0.100707 -0.463758  0.087314  0.292798  0.137456  
             20231101  0.001377 -0.059778  0.016609  0.050532  0.017220  
             20231102  0.002128 -0.037798  0.009851  0.036596  0.031129  
...                         ...       ...       ...       ...       ...  
             20240402 -0.547327 -1.878498  0.699713  1.178075  0.913404  
             20240403 -0.641482 -1.437312  0.432396  0.983013  0.741509  
             20240404 -0.154793 -0.368110  0.174599  0.251095  0.174101  
             20240405 -0.008176 -0.378641  0.112632  0.164001  0.123232  
             20240406 -0.175339 -0.509901  0.121158  0.385448  0.128329  

[86 rows x 1536 columns]

In [80]:
import tushare as ts
pro = ts.pro_api()
for i, g in df_embeddings.groupby('code'):
    print(g['code'].iloc[0])
    code = g['code'].iloc[0]
    part1 = code[:2]
    part2 = code[2:]

    idx = dict_arr[code]
    sorted_g = g.sort_values(by='date')
    sorted_g.insert(2,'idx',idx)
    df_day = sorted_g['date'].apply(lambda x: x.strftime('%Y%m%d'))
    min_date = (df_day.min())
    max_date = (df_day.max())
    
    df_kline = pro.daily(ts_code=part2+'.'+part1, start_date=min_date, end_date=max_date)
    # df_day left join df_kline
    df_day = df_day.to_frame()
    df_day.columns = ['trade_date'] 
    df_day = df_day.merge(df_kline, on='trade_date', how='left')

    # cat two dataframe horizontally one is sorted_g and the other is df_day    
    df_all = pd.concat([ sorted_g.reset_index(drop=True), df_day.reset_index(drop=True)], axis=1)
    
    #df_day = df_day.merge(sorted_g, on='idx', how='left')
    #break
    df_all.drop(['trade_date','ts_code'], axis=1, inplace=True)
    df_all.dropna(inplace=True)
    df_all.to_csv(f'./index_300/{code}.csv',index=False)

sh600036
sh600276
sh600519
sh600900
sh601166
sh601318
sh601899
sz000333
sz000858
sz300750


In [78]:
df_all[-6:]

,date,code,idx,0,1,2,3,4,5,6,...,1535,open,high,low,close,pre_close,change,pct_chg,vol,amount
960,2024-04-06 06:39:00,sz300750,0,0.118040,-0.111507,-0.018272,0.080883,0.040546,0.069117,-0.008908,...,0.005173,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
961,2024-04-06 06:39:00,sz300750,0,0.059917,-0.104301,0.056999,0.100061,0.055509,0.009099,0.043320,...,0.058387,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
962,2024-04-06 08:06:00,sz300750,0,0.051554,-0.090702,0.075059,0.056664,0.013563,-0.036048,0.038099,...,0.024600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
963,2024-04-06 09:22:00,sz300750,0,0.028303,-0.061776,0.102021,0.047699,-0.015367,-0.013320,0.010171,...,0.001846,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
964,2024-04-06 10:46:00,sz300750,0,-0.059058,0.076216,0.073836,0.230623,0.020653,0.170882,-0.000234,...,-0.082382,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
965,2024-04-06 14:09:00,sz300750,0,0.037437,0.023296,-0.032922,0.126579,-0.027781,-0.049125,0.005928,...,-0.010793,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [73]:
#df_all.drop(['trade_date','ts_code'], axis=1, inplace=True)
df_all.columns.to_list()[-40:]

[1505,
 1506,
 1507,
 1508,
 1509,
 1510,
 1511,
 1512,
 1513,
 1514,
 1515,
 1516,
 1517,
 1518,
 1519,
 1520,
 1521,
 1522,
 1523,
 1524,
 1525,
 1526,
 1527,
 1528,
 1529,
 1530,
 1531,
 1532,
 1533,
 1534,
 1535,
 'open',
 'high',
 'low',
 'close',
 'pre_close',
 'change',
 'pct_chg',
 'vol',
 'amount']

In [41]:
print(pd.concat([df_day.reset_index(drop=True), sorted_g.reset_index(drop=True)],axis=1).iloc[7][:])

trade_date     20231030
ts_code       600036.SH
open              31.01
high              31.13
low               30.55
                ...    
1531           0.021719
1532          -0.047426
1533           0.007777
1534           0.055747
1535           0.014372
Name: 7, Length: 1550, dtype: object


In [11]:
sorted_g['date'].apply(lambda x: x.strftime('%Y%m%d'))

2032    20231028
2033    20231029
2034    20231029
2031    20231029
2035    20231029
          ...   
3967    20240406
3968    20240406
3969    20240406
3970    20240406
3971    20240406
Name: date, Length: 978, dtype: object

In [ ]:
import tushare as ts
pro = ts.pro_api()

df = pro.daily(ts_code='000001.SZ', start_date='20180701', end_date='20180718')